In [2]:
### Import external modules used in the paper
import pandas as pd
import numpy as np
from scipy.stats import gamma, chi2
import itertools
# This is the only non-standard module used here. We only use it to implement false-discovery rate correction
from statsmodels.sandbox.stats.multicomp import fdrcorrection0 as fdr 

In [3]:
### This code takes in a data set that consists of p-values for the differential expression of each
### orthology group in each organism undergoing regeneration. The bulk of those code is there to process
### the data and convert it into a Pandas data frame that is amenable to efficient statistical analysis.
### The main piece of analysis in this cell performs p-value aggregation using the Lancaster method.
### This cell initializes the library of custom functions that are used in later cells


# This function takes in the text file containing RNA-seq data that contains (log) read counts and p-values for time series from each gene in each organism we use
# for the analysis. 
def read_file(folder, file):
    """
    This function takes in a text file containg log_2 read counts and p-values from RNA-seq time series and converts it to a Pandas data frame.
    This function is intended to be fairly generic, while the internal function read_file() is meant to be specialized to the particular format of the input data.
    Args:
        folder (string): Folder where the text file is located and where we will store saved data.
        file (string): Text file with RNA-seq data.
    Returns:
        dict: Dictionary where keys corresponding to the differnet organisms we compare, and values consisiting of (gene, ortholog group, read count, p-value) data.
        
    """
    data = {} # initialize output. This is a dictionary that will be modeled after a json-like structure. 
    with open(folder + file) as f:
        content = f.readlines()
        
        for line in content:
            row = line.rstrip().split() # just some simple formating and removal of whitespace
            if row: # this block just checks to make sure the row isn't empty, and gets rid of some elements that aren't necessary for the purposes of this code.
                try:
                    del row[1]
                except Exception as e:
                    print(row)
                org = row[1]
                
                if org not in data.keys(): # initialize new entry when we get to a new organism
                    data[org] = []
                    
                reads = [float(el) for el in row[3::2]] # log_2 read counts for the gene
                pvals =  [float(el) for el in row[4::2]] # p-values for the gene
                
                if len(pvals) > 0:
                    entry = {'gene': row[0], 'ogroup': row[2], 'reads':reads, 'pvals':pvals}
                    data[org].append(entry)
    return data
    

def file2df(folder,file):
    """
    This function takes in a text file containg log_2 read counts and p-values from RNA-seq time series and converts it to a Pandas data frame.
    This function is intended to be fairly generic, while the internal function read_file() is meant to be specialized to the particular format of the input data.
    Args:
        folder (string): Folder where the text file is located and where we will store saved data.
        file (string): Text file with RNA-seq data.
    Returns:
        DataFrame: Pandas data frame on which we perform statistical analysis
        
    """
    data = read_file(folder,file) 
    data_agg = []
    
    for key in data.keys():
        for row in data[key]:
            if row['pvals']:
                row['organism'] = key
                data_agg.append(row)
        print(len(data_agg))
    df = pd.DataFrame(data_agg)
    return df


def lancaster(row):
    """
    This is a Python implementation of the Lancaster method for p-value aggregation. This implementation is based on the method described in
    Gene-level differential analysis at transcript-level resolution, Yi et al. 2018, Genome Biology.
    Args:
        row (pd.Series): a row of a data frame consisting of a list of p-values and log_2 read counts
    Returns:
        p (float): aggregated p-value
    """
    tau = 0
    pvals = row['pvals']
    reads = [2**r for r in row['reads']]
    t = [gamma.isf(max(p,tau),w/2, scale=2) for p, w in zip(pvals,reads)]
    t = sum(t)
    p = chi2.sf(t,sum(reads))
    return p


def overlap_subsets(Ogroup_agg):
    """
    This function takes in a data frame consisting of ortholog groups from each organism with their associated lancaster p-values.
    It then adjusts the p-values to account for false-discovery rate corrections. It then uses these final p-values to determine which orthology groups are present
    across each possible subset of organisms for which we have data.
    Args:
        Ogroup_agg (pd.DataFrame): Data frame that comes from the
    Returns:
         Data Frame: Data frame consisting of each possible combination of organisms and the ortholog groups that pass significance test within the subset
    """
    og_lists = {}
    agg_list = []
    # false-discovery rate correction applied to list of lancaster p-values computed for ortholog groups for each organism. the function fdr() returns the orthology groups which
    # have adjusted p-value > 0.05.
    for name, group in Ogroup_agg.groupby('organism'): 
        padj = fdr(group.p_lanc,alpha = .05) 
        og_l = list(group.ogroup.loc[padj[0]]) # list of ortholog groups for a given organism that pass significance test
        og_lists[name] = og_l
        agg_list += og_l
    agg_list = np.unique(agg_list)

    overlap = {}
    for og in agg_list: # iterate over every ortholog group and track which organisms 
        overlap[og] = []
        for key in og_lists:
            if og in og_lists[key]:
                overlap[og].append(key)

    ol_subsets = dict()
    orgs = Ogroup_agg.organism.unique()
    
    for i in range(orgs.size): # iterate over every possible subset of organisms and compile list of ortholog groups that pass significance threshold in each organism in a given subset
        for el in itertools.combinations(orgs, i+1): 
            ol_subsets[str(list(el))] = pd.Series([og for og in agg_list if set(el).issubset(overlap[og])])
            
    return pd.DataFrame.from_dict(ol_subsets)


In [6]:
### This cells runs the analysis that takes in a text file with RNA-seq data and outputs a Pandas data frame and computes aggregated p-values for each orthology group. 

# file = '1_Annotated_Pvalue_Matrix.6-2-18.txt'
# folder = '06-07-18_Data/'
file = '1_Annotated_Pvalue_Matrix.HeatShock.9-21-2020.txt'
folder = '09-30-20_Data/'
df = file2df(folder, file)
df.to_csv(folder + 'pcounts.csv',index=False) #saves the data frame version of the text file

# Some reorganization of the data frame so that we aggregate all paralogs from the same ortholog group. This implicitly assumes
# that these p-values can be treated as independent test of the differnetial expression hypothesis.
Ogroup_agg = df.groupby(['organism','ogroup'],as_index=True)['pvals','reads'].sum() 
 
Ogroup_agg['p_lanc'] = Ogroup_agg.agg(lancaster,axis=1) # apply the Lancaster method to each orthology group. This step can take a few minutes!
Ogroup_agg.to_csv(folder + 'lanc.csv') # saves the aggregated data frame with the lancaster p-values

15774
64952
115533
164238
189106
222860
238745


In [7]:
A = pd.read_csv(folder + 'lanc.csv')
overlap_df = overlap_subsets(A)
overlap_df.to_csv(folder + 'ol_subsets.csv')